# Loss для детектора

Как подсчитать loss для детектора. Loss должн включать в себя две части: ошибку локализации и ошибку классификации.

И для SSD loss function так и выглядит:

$$\large L(x,c,l,g) = \frac{1}{N}(L_{conf}(x,c) + \alpha L_{loc}(x,l,g))$$


Однако если мы будем считать ошибку локализации для всех default box, то она никогда не будет нулевой.
Default box очень плотно перекрывает все изображение, и в большинство из них объект не попадет, особенно если объект один и небольшой.

<center><img src ="http://edunet.kea.su/repo/EduNet-additions/L11/default_boxes.png" width="700"></center>

<center><em>Source: <a href="https://ternak.github.io/cnn_detection.html">Object Detection With Convolution Neural Nets</a></em></center>

Поэтому часть default box при подсчете loss игнорируются. Используются только те, у которых площадь пересечения с одним из истинных bounding box больше порога ($\text{IoU} > 0.5$)



$$\large L(x,l,g)_{loc} = \sum_{i \in Pos}^{N} x_{i,j}^{k}smooth_{L1}(l_i, g_j)$$


Здесь:

$l$ — финальные координаты предсказанного bounding box с учетом смещений,

$g$ — координаты истинного bounding box,

$M$ — количество истинных (ground true) bounding box-ов,

$Pos$ — список отобранных default box, пересекающихся с истинными,

$x_{i,j}^{k} = \{1,0\}$ — индикатор того, что комбинация default и box валидна.



> $i$ — индекс default box,
> $j$ — индекс истинного (ground true) bounding box,
> $p$ — номер класса, к которому относится ground true bounding box (не степень).

$smooth_{L1}$ — [Комбинация L1 и L2](https://pytorch.org/docs/stable/generated/torch.nn.SmoothL1Loss.html)

Компонент, отвечающий за классификацию:

$$\large L(x,с)_{conf} = -\sum_{i \in Pos} x_{i,j}^{k} log(softmax(c_{i}^{p})) -\sum_{i \in Neg} log(softmax(c_{i}^{0}))$$

$c_{i}^{p}$ — вектор score для $i$-того default box, $p$ — номер истинного класса, соответствующего bounding box из разметки

$Pos$ — список отобранных default box, не пересекающихся с истинными ($\text{IoU} < threshold$)


\* *Формулы для loss function осознанно упрощены. Например, мы опустили расчет L1 для смещений, что является технической деталью.*

# FocalLoss

Следующий заслуживающий внимания one-stage детектор — это RetinaNet ([Focal Loss for Dense Object Detection (Lin et al., 2017)](https://arxiv.org/abs/1708.02002)).

Собственно, авторы придумали новую функцию потерь (Focal Loss) и опубликовали модель, чтобы продемонстрировать её эффективность.

Чтобы понять, какую проблему решает Focal Loss, давайте посмотрим на второй компонент Loss классификации для SSD:

$$\large L_{conf} =  \ ...\  -\sum_{i \in Neg} log(softmax(c_{i}^{0}))$$

Это кросс-энтропия для bounding boxes, содержащих фон. Тут нет ошибки: когда модель обучится правильно предсказывать класс фона (background), каждая из этих компонент будет небольшой.

Проблема в том, что таких компонент очень много. Детектор предсказывает несколько тысяч, или десятков тысяч bounding boxes. Подавляющая часть из них приходится на фон. Cумма большого количества этих небольших потерь (loss) становится заметным числом и мешает учиться классифицировать реальные объекты.

Как решается эта проблема в Focal Loss?

<center><img src ="https://ml.gan4x4.ru/msu/additions/L11/focal_loss_vs_ce.png" width="700"></center>

<center><em>Source: <a href="https://arxiv.org/abs/1708.02002">Focal Loss for Dense Object Detection (Lin et al., 2018)</a></em></center>



Фактически loss для уверенно классифицированных объектов дополнительно занижается. Это похоже на взвешивание при дисбалансе классов.

Достигается этот эффект путем домножения на коэффициент: $ (1-p_{t})^\gamma$

Здесь:

$ p_{t} $ — вероятность истинного класса,

$ \gamma $ — число больше $1$, являющееся гиперпараметром.

Пока модель ошибается, $p_{t}$ — мало, и значение выражения в скобках соответственно близко к $1$.

Когда модель обучилась, значение $p_{t}$ становится близким к $1$, а разность в скобках становится маленьким числом, которое возводится в степень $ \gamma > 1$. Таким образом, домножение на это небольшое число нивелирует вклад верно классифицированных объектов.

Это позволяет модели сосредоточиться на изучении сложных объектов (hard examples)

[Подробнее про FocalLoss](https://github.com/Gan4x4/ml_snippets/blob/main/FocalLoss.ipynb)

# Нard Example Mining

При обучении модели мы можем обнаружить, что средняя ошибка на всех данных достаточно маленькая, однако ошибка на редких нетипичных данных довольно высока. При этом нетипичные данные необязательно являются выбросами.

Разберемся, почему так происходит.
К примеру, рассмотрим задачу обнаружения автомобилей на потоках данных с камер наружного видеонаблюдения. Если в обучающем наборе большая часть данных — снимки, сделанные днём, то качество работы модели ночью будет низким. В данном случае, "нетипичными" данными будут ночные снимки. Но на самом деле "нетипичных" случаев может быть довольно много, и некоторые из них могут происходить даже днём. Например:
* изменение погоды (изменение яркости, резкости, помехи на изображении),
* смена сезона (снег либо листья могут покрыть дорогу — изменение фона),
* машины с экзотическими узорами на кузове.

Довольно простым и эффективным решением проблемы является сбор "сложных" случаев (**hard example mining**) и дообучение модели на них. При этом, поскольку модель уже довольно хорошо работает на большей части данных, можно дополнительно удалить часть данных из обучающей выборки — таким образом мы сосредотачиваем модель на обучении на сложных примерах.

<img src ="https://ml.gan4x4.ru/msu/additions/L11/hard_example_mining.png" width="1000">

## Online hard example mining

В некоторых случаях hard exapmle mining можно выполнять прямо во время формирования батча, "налету". В таких случаях говорят про **online hard example mining**.

Один из вариантов может быть реализован в two-stage детекторах.
Напоминаю: первая часть детектора отвечает за обнаружение regions of interest (ROI), затем выполняется (как правило, сравнительно вычислительно дешёвая) классификация. Одним из вариантов реализации идеи может быть выполнение forward pass классификатора по всем предложенным ROI и затем формирование батча, в котором будет выделено определённое количество "мест" под ROI, предсказания на которых выполняются наихудшим образом.

<center><img src ="https://ml.gan4x4.ru/msu/additions/L11/online_hard_example_mining.png" width="700"></center>

<center><em>Source: <a href="https://sklin93.github.io/hog.html"> HoG Face Detection with a Sliding Window </a></em></center>



# Feature pyramid network

Вторым полезным нововведением  в RetinaNet стало использование пирамиды признаков.

[Feature Pyramid Networks for Object Detection (Tsung-Yi Lin et al., 2017)](https://arxiv.org/abs/1612.03144)

<center><img src ="https://ml.gan4x4.ru/msu/additions/L11/retinanet_use_outputs_fpn.png" width="900"></center>

<center><em>Source: <a href="https://link.springer.com/article/10.1007/s11042-022-13153-y?error=cookies_not_supported&code=d283d48a-d725-4d7e-a568-7955a14a0550">Tools, techniques, datasets and application areas for object detection in an image: a review</a></em></center>


RetinaNet использует выходы FPN для предсказаний и класса, и bbox. Мы уже обсуждали пирамиды признаков применительно к сетям для сегментации, в частности, FCN.

На каждом сверточном слое извлекаются карты признаков.

Их пространственное разрешение постепенно уменьшается, а глубина (количество каналов) увеличивается.


Но первые слои содержат мало семантической информации (только низкоуровневые признаки).  А карты признаков с глубоких слоев имеют низкое пространственное разрешение, что не позволяет качественно определить границы объектов.





<center><img src ="https://ml.gan4x4.ru/msu/additions/L11/semantic_information.png" width="650"></center>

<center><em>Source: <a href="https://arxiv.org/pdf/1612.03144.pdf">Feature Pyramid Networks for Object Detection</a></em></center>

Так же, как и в случае с сегментацией, точность повышается, если делать предсказания на картах, содержащих признаки для разных масштабов.

При этом можно получать карты с большим пространственным разрешением не просто сохраняя их в памяти, но еще и прибавляя к ним значения признаков с более глубоких слоев, предварительно интерполировав их (Upsample).

Идея состоит в том, чтобы делать предсказание с учетом семантической информации, полученной на более глубоких слоях. Здесь признаки   суммируются, а не конкатенируются.

Затем к новым картам признаков может применяться дополнительная свертка.

На выходе получаем карты признаков P2–P5, на которых уже предсказываются bounding boxes.

<img src ="https://ml.gan4x4.ru/msu/additions/L11/resnet_prediction_head_scheme.png" width="850">

В случае 2-stage детектора (RCNN) новые карты признаков подаются на вход RPN.

<center><img src ="http://edunet.kea.su/repo/EduNet-additions/L11/features_from_blackbone.jpeg" width="1100"></center>

<center><em>Source: <a href="https://jonathan-hui.medium.com/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c">Understanding Feature Pyramid Networks for object detection (FPN)</a></em></center>

А признаки для предсказаний используются из backbone.

Дополнительно: [Блог-пост про FPN](https://jonathan-hui.medium.com/understanding-feature-pyramid-networks-for-object-detection-fpn-45b227b9106c)